### Header Code

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
import plotly.graph_objs as go
import math
import time
from scipy.optimize import fmin
from scipy.optimize import minimize

## BAPM Functions

In [48]:
#generates the stock price vector for the BAPM function
def generate_stockPrices(u, d, row, col, matrix):
    if row > col:
        return matrix
    if ((row == 0) and (col == 0)):
                matrix[row][col] = 1
    else:
        matrix[row][col] = u**(col - row) * d**(row)
    return matrix

In [47]:
#generates the state price vector for the BAPM function
def generate_statePrices(q, row, col, matrix):
    if col == row:
        matrix[row][col] = q    
    if col - row == -1:
        matrix[col][row] = 1-q
    return matrix

In [46]:
def generate_finalValue(call, strikePrice, value):
    if call:
        value = value - strikePrice
    else:
        value = strikePrice - value
    return np.clip(value, a_min=0, a_max=None)

In [45]:
def generate_optionPrices(optionType, strikePrice, valueMatrix, priceMatrix, q, discount):
    #print(valueMatrix.shape[1])
    for j in range(valueMatrix.shape[1]-1, 0, -1):#loop backwards through columns first
        for i in range(1, valueMatrix.shape[0]):#loop downwards through rows; saves final spot of [0,0]
            if i > j:
                continue
            if not optionType:#european option pricing
                valueMatrix[i-1][j-1] = discount*(q*valueMatrix[i-1][j] + (1-q)*valueMatrix[i][j])
                
            if optionType:#american option pricing
                continiousValue = discount*(q*valueMatrix[i-1][j] + (1-q)*valueMatrix[i][j])
                exerciseValue = priceMatrix[i-1][j-1] - strikePrice
                valueMatrix[i-1][j-1] = max(exerciseValue, continiousValue)
                
    return valueMatrix

In [44]:
def bapm(curPrice, strikePrice, volatility, monthTerm, riskFree, numPeriods):
    print('0: Put\n1: Call')
    
    call = None
    while call not in [0, 1]:
        call = int(input('Select a 0 or 1: '))
    
    print('\n0: European\n1: American')
    optionType = None
    while optionType not in [0, 1]:
        optionType = int(input('Select a 0 or 1 for option type: '))
        
    start = time.clock()
    #setting variables
    monthTerm = monthTerm/12
    timeStep = monthTerm/numPeriods
    #volatility = np.sqrt((volatility**2)/(12/timeStep))#scaling the volatility based on timeStep and months term for the option
    #not sure which time frame for vol to use, 3 months or 6 months
    u = np.exp(volatility*(np.sqrt(timeStep)))
    d = u**-1
    q = (np.exp(riskFree*timeStep) - d)/(u-d)
    discount = np.exp(-riskFree*timeStep)
    
    priceMatrix = np.zeros((numPeriods+1, numPeriods+1))
    statePrices = np.zeros((numPeriods+1, numPeriods+1))
    
    for i in range(0, numPeriods + 1):#row loop
        for j in range(0, numPeriods + 1):#col loop
            #I chose to loop through the cols first and then rows
            statePrices = generate_statePrices(q=q, row=i, col=j, matrix=statePrices)
            priceMatrix = generate_stockPrices(u=u, d=d, row=i, col=j, matrix=priceMatrix)
    
    priceMatrix = curPrice*priceMatrix
    
    valueMatrix = np.zeros((numPeriods+1, numPeriods+1))
    valueMatrix[:, -1] = generate_finalValue(call=call, strikePrice=strikePrice, value=priceMatrix[:, -1])
    
    optionPrices = generate_optionPrices(optionType=optionType, strikePrice=strikePrice,
                                         valueMatrix=valueMatrix, priceMatrix=priceMatrix,
                                         q=q, discount=discount)
    
    #############################################################################################################
    #TESTING FUNCTIONS
    #print('Price Matrix:\n', priceMatrix, '\n')
    #print('Value Matrix:\n', valueMatrix)
    #print('State Prices Matrix:\n', statePrices, '\n')
    #print('Option Value --  State Prices Method --:\n', np.dot((discount*statePrices)**numPeriods, valueMatrix[:, -1]), '\n')
    print('Option Values -- backwards loop --:\n', optionPrices)
    #########################################################################################################
    end = time.clock()
    #print("Time Elapsed: ", end-start)
    
    return optionPrices[0][0]

In [50]:
bapm(strikePrice=40, curPrice=40, volatility=.3, monthTerm=6, riskFree=.04, numPeriods=50)

0: Put
1: Call
Select a 0 or 1: 0

0: European
1: American
Select a 0 or 1 for option type: 1
Option Values -- backwards loop --:
 [[  6.67901742   6.87074387   7.20360213 ... 128.84382948 133.96940564
    0.        ]
 [  0.           6.49326372   6.54448066 ... 119.0120619  123.83821616
    0.        ]
 [  0.           0.           6.44740405 ... 109.75285189 114.29702123
    0.        ]
 ...
 [  0.           0.           0.         ...  30.49090245  30.21827187
   29.93685788]
 [  0.           0.           0.         ...   0.          30.78698379
   30.52288965]
 [  0.           0.           0.         ...   0.           0.
   31.07479359]]


6.679017423845292

In [26]:
def findVol(vol):
    calculatedPrice = bapm(strikePrice=40, curPrice=40, volatility=vol, monthTerm=6, riskFree=.04, numPeriods=2)
    realPrice = 3.37398
    
    return calculatedPrice - realPrice

In [27]:
parameters = [.3]
res = minimize(fun=findVol, x0=parameters)

Option Value --  State Prices Method --:
 [3.37391914 0.         0.        ] 

Option Values -- backwards loop --:
 [[ 3.37391914  6.87137636 13.9943523 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Value --  State Prices Method --:
 [3.37391928 0.         0.        ] 

Option Values -- backwards loop --:
 [[ 3.37391928  6.87137671 13.99435311]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Value --  State Prices Method --:
 [3.37391914 0.         0.        ] 

Option Values -- backwards loop --:
 [[ 3.37391914  6.87137636 13.9943523 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
Option Value --  State Prices Method --:
 [ 0.          7.35828629 13.35566098] 

Option Values -- backwards loop --:
 [[ 7.35828629  0.          0.        ]
 [ 0.         17.44523282  0.        ]
 [ 0.          0.         41.35965035]]
Option Value --  State Prices Method --:
 [ 0.          7.358

In [28]:
print('Parameters: ', res.x)

Parameters:  [0.01999999]
